In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
tabla_historico = pd.read_excel('historica_Iberdrola_DIC_16-19.xls')

In [6]:
tabla_historico

Fecha  Último
0    29-nov-2019   8.928
1    28-nov-2019   8.916
2    27-nov-2019   8.928
3    26-nov-2019   8.896
4    25-nov-2019   8.898
..           ...     ...
735  07-dic-2016   5.729
736  06-dic-2016   5.775
737  05-dic-2016   5.581
738  02-dic-2016   5.599
739  01-dic-2016   5.599

[740 rows x 2 columns]

In [7]:
# Division of a global sequences into n_step size ones (X) with the prediction desired (y)
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [8]:
# Calculates a successfull ratio using less relative error than a maximum criterium
import keras.backend as K
def porcentaje_margen(y_true, y_pred):
    margen = 0.02
    yy = K.sum(K.cast(K.less(K.abs((y_pred/y_true)-1.0), margen), dtype=float))
    return yy/K.cast(K.shape(y_pred)[0], dtype=float)


In [9]:
XMAX = 15.0
T = 20
X_data, y_data = split_sequence(sequence=tabla_historico['Último'], n_steps=T)
X = X_data/XMAX
y = y_data/XMAX
n_features = 1 #Nº atributos

X.shape

(720, 20)

In [10]:
X = X.reshape((X.shape[0], X.shape[1], n_features))
X.shape

(720, 20, 1)

In [11]:
P = y.shape[0]
a = np.arange(P, dtype=int)
np.random.shuffle(a)
sequences_train = a[0:int(P*2/3)]
sequences_test = a[int(P*2/3):]

X_train = X[sequences_train]
X_test = X[sequences_test]
y_train = y[sequences_train]
y_test = y[sequences_test]

In [ ]:
model = Sequential()
model.add(SimpleRNN(10, return_sequences=True, input_shape=(T, n_features)))
model.add(SimpleRNN(5, return_sequences=False))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])

In [ ]:
history=model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=200,verbose=1 ,steps_per_epoch=48, validation_steps=1)

In [ ]:
history.epoch

In [ ]:
history.history['porcentaje_margen']

In [ ]:
plt.plot(history.epoch, history.history['val_porcentaje_margen'], label='validation')
plt.plot(history.epoch, history.history['porcentaje_margen'], label='training')
plt.legend()
plt.title('Success Rate Evolution Simple RNN')
plt.xlabel('Epoch')
plt.show()

In [ ]:
model = Sequential()
model.add(LSTM(5, return_sequences=False, input_shape=(T, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])

In [ ]:
history=model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=200,verbose=1, steps_per_epoch=48, validation_steps=1)

In [ ]:
plt.plot(history.epoch, history.history['val_porcentaje_margen'], label='validation')
plt.plot(history.epoch, history.history['porcentaje_margen'], label='training')
plt.legend()
plt.title('Success Rate Evolution LSTM')
plt.xlabel('Epoch')
plt.show()

In [ ]:
model = Sequential()
model.add(GRU(5, return_sequences=False, input_shape=(T, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])

In [ ]:
history=model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=200,verbose=1, steps_per_epoch=48, validation_steps=1)

In [ ]:
plt.plot(history.epoch, history.history['val_porcentaje_margen'], label='validation')
plt.plot(history.epoch, history.history['porcentaje_margen'], label='training')
plt.legend()
plt.title('Success Rate Evolution Simple GRU')
plt.xlabel('Epoch')
plt.show()